In [1]:
from pyspark.sql.types import *
from pyspark.sql import Row

# Introduction
We are going to use the dating profiles dataset for this tutorial. More information on this dataset can be found [here](https://sites.google.com/a/insightdatascience.com/spark-lab/s3-data/dating-profiles).

# Loading in the data
First, read in the data into an RDD. Here, we are using 16 partitions for our RDD.

In [2]:
# Read in raw ratings data (fromUserId, toUserId, rating)
ratingsCsvRDD = sc.textFile("s3a://insight-spark-after-dark/ratings.csv.gz").repartition(6)

Reformat the data into a JSON format by applying the function rec_tup (which converts the data into a JSON format) to the RDD using the .map and .toDF() transformation steps.

In [4]:
# Convert raw ratings RDD to Json RDD
def rec_tup(rating):
    tokens = rating.split(",")
    return Row(fromUserId=int(tokens[0]), toUserId=int(tokens[1]), rating=float(tokens[2]))

ratingsJson_DF = ratingsCsvRDD.map(rec_tup).toDF()
ratingsJson_DF.take(5)

[Row(fromUserId=1, rating=7.0, toUserId=1978),
 Row(fromUserId=1, rating=5.0, toUserId=4842),
 Row(fromUserId=1, rating=10.0, toUserId=9345),
 Row(fromUserId=1, rating=4.0, toUserId=12638),
 Row(fromUserId=1, rating=9.0, toUserId=15123)]

As you might remember, Spark does a Lazy evaluation which means each transformed RDD may be recomputed each time you run an action on it. We'll use the .persist method (this is actually also a transformation) to keep the result (transformed RDD) on the cluster for quick future access.

**Note**: persist is a transformation, it will only run when a future action is called. One result of this is that if you run an action twice, you will persist twice (and right now in Spark you will lose the pointer to the first persist).

In [5]:
# Cache the SchemaRDD as we'll be using this heavily moving forward
ratingsJson_DF.persist(StorageLevel.MEMORY_AND_DISK_SER)

DataFrame[fromUserId: bigint, rating: double, toUserId: bigint]

Quickly check the schema of the dataframe. 
To write SparkSQL, we need to create a table object from our dataframe which we can use to run SparkSQL commands. The transformation registerTempTable() does this and we call our table 'ratingsJsonTable'.

In [6]:
# Describe the SchemaRDD inferred from the JSON
ratingsJson_DF.printSchema()

ratingsJson_DF.registerTempTable("ratingsJsonTable")

root
 |-- fromUserId: long (nullable = true)
 |-- rating: double (nullable = true)
 |-- toUserId: long (nullable = true)



# On to SparkSQL

SparkSQL is an abstract API that lets you use regular SQL commands to query your Spark cluster.

Now that we have created a table of our data we can call SQL commands. 

**Note:** .sql is a transformation and we need to call .collect() to execute the job.

In [7]:
# Details of a table
sqlContext.sql("DESCRIBE ratingsJsonTable").collect()

[Row(col_name=u'fromUserId', data_type=u'bigint', comment=u''),
 Row(col_name=u'rating', data_type=u'double', comment=u''),
 Row(col_name=u'toUserId', data_type=u'bigint', comment=u'')]

In [8]:
# Show the top 10 most-active users who are giving out ratings
mostActiveUsersSchemaRDD = sqlContext.sql("""
    SELECT fromUserId, count(*) AS num_rated
    FROM ratingsJsonTable 
    GROUP BY fromUserId
    ORDER BY num_rated DESC
    LIMIT 10
    """)

In [9]:
mostActiveUsersSchemaRDD.collect()

[Row(fromUserId=90280, num_rated=25042),
 Row(fromUserId=56792, num_rated=21599),
 Row(fromUserId=33639, num_rated=19908),
 Row(fromUserId=61436, num_rated=18810),
 Row(fromUserId=72351, num_rated=18443),
 Row(fromUserId=127227, num_rated=18342),
 Row(fromUserId=58765, num_rated=18197),
 Row(fromUserId=76082, num_rated=18019),
 Row(fromUserId=108318, num_rated=17755),
 Row(fromUserId=131976, num_rated=17560)]

# Next Steps

Here are some further questions to get to grips with SparkSQL.

### Task 1: Who are the top 10 users that have the biggest variance in ratings of their profile? 

In [ ]:
largestVarianceSchemaRDD = sqlContext.sql("""
    SELECT toUserId, variance(rating) AS var_rating
    FROM ratingsJsonTable 
    WHERE rating > 0
    GROUP BY toUserId
    ORDER BY var_rating DESC
    LIMIT 10
    """)
largestVarianceSchemaRDD.collect()

### Task 2: How many pairs of users have rated each other with a rating > 5?

In [11]:
pairsRating5SchemaRDD = sqlContext.sql('''
select
    a.fromUserId,
    a.toUserId,
    a.rating as rating_forward,
    b.rating as rating_backwards
from
    ratingsJsonTable a
join
    ratingsJsonTable b
on 
    a.fromUserId = b.toUserId
    and a.toUserId = b.fromUserId
where
    a.rating > 5
    and b.rating > 5
''')
pairsRating5SchemaRDD.take(10)

[Row(fromUserId=48, toUserId=65571, rating_forward=10.0, rating_backwards=10.0),
 Row(fromUserId=77, toUserId=81498, rating_forward=10.0, rating_backwards=10.0),
 Row(fromUserId=586, toUserId=59865, rating_forward=10.0, rating_backwards=10.0),
 Row(fromUserId=979, toUserId=41724, rating_forward=7.0, rating_backwards=8.0),
 Row(fromUserId=1055, toUserId=1712, rating_forward=10.0, rating_backwards=10.0),
 Row(fromUserId=1541, toUserId=83330, rating_forward=10.0, rating_backwards=10.0),
 Row(fromUserId=1895, toUserId=131432, rating_forward=10.0, rating_backwards=10.0),
 Row(fromUserId=2383, toUserId=15976, rating_forward=10.0, rating_backwards=10.0),
 Row(fromUserId=2383, toUserId=91376, rating_forward=10.0, rating_backwards=10.0),
 Row(fromUserId=2524, toUserId=51359, rating_forward=8.0, rating_backwards=8.0)]

In [ ]:
print "{} pairs of users".format(pairsRating5SchemaRDD.count())